In [ ]:
# ======== COMBINED API DATA FETCHER FOR GOOGLE COLAB ========
# This file combines Kaggle API setup with Exchange Rate, Alpha Vantage, and World Bank data fetching

import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from kaggle.api.kaggle_api_extended import KaggleApi

# ======== API CONFIG (REPLACE THESE!) ========
EXCHANGE_API_KEY = "1e4a5a207d2743f034b711fa"       # From ExchangeRate-API (or leave blank for Frankfurter)
ALPHA_VANTAGE_API_KEY = "XIFV0Y7PVGHFVYPN" # From https://www.alphavantage.co/
WORLD_BANK_COUNTRY = "US"                       # ISO3 country code (e.g., "US", "CN")
WORLD_BANK_INDICATOR = "NY.GDP.MKTP.CD"         # GDP in USD (see https://data.worldbank.org/indicator)
STOCK_SYMBOL = "IBM"                            # Stock or crypto symbol (e.g., "BTC", "TSLA")

# ======== KAGGLE API SETUP ========
def setup_kaggle_api():
    """Setup and authenticate Kaggle API"""
    print("=== 🔧 KAGGLE API SETUP ===")

    # --- Step 1: Clean up and Upload ---
    # This ensures we start with a clean slate.
    !rm -f kaggle*.json
    !rm -rf ~/.kaggle

    print("Please upload your kaggle.json file:")
    try:
        uploaded = files.upload()
    except Exception as e:
        print(f"Upload failed or was cancelled: {e}")
        # Stop execution if the upload fails
        raise SystemExit("Upload is required to proceed.")

    # --- Step 2: Automatically Configure the Uploaded File ---
    try:
        # This automatically gets the name of the file you just uploaded.
        uploaded_filename = list(uploaded.keys())[0]
        print(f'Successfully uploaded file: "{uploaded_filename}"')

        # Create the target directory.
        !mkdir -p ~/.kaggle

        # Move the uploaded file to the target directory AND rename it to 'kaggle.json'.
        # The quotes handle any special characters in the filename.
        !cp '{uploaded_filename}' ~/.kaggle/kaggle.json

        # Set the required file permissions.
        !chmod 600 ~/.kaggle/kaggle.json

        print("Successfully configured the Kaggle API key.")

    except Exception as e:
        print(f"An error occurred during file configuration: {e}")
        # Stop execution if configuration fails
        raise SystemExit("Configuration failed.")

    # --- Step 3: Authenticate and Verify ---
    try:
        print("\nAttempting to authenticate...")
        api = KaggleApi()
        api.authenticate()
        print("✅ Authentication Successful!")

        # --- MODIFIED PART ---
        # Verify by searching for datasets related to 'exchange rate'.
        print("\nVerification: Listing datasets related to 'exchange rate'...")
        !kaggle datasets list -s "exchange rate"

        return api

    except Exception as e:
        print(f"\n❌ Authentication Failed: {e}")
        return None

# ======== 1. WORLD BANK API (ECONOMIC DATA) ========
def fetch_world_bank_data(country, indicator, year="2022"):
    """Fetch economic data from World Bank API"""
    url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1 and data[1]:
            value = data[1][0]['value']
            return {
                "indicator": data[1][0]['indicator']['value'],
                "country": data[1][0]['country']['value'],
                "year": year,
                "value": f"${value:,.2f}" if indicator == "NY.GDP.MKTP.CD" else value
            }
    return {"error": "Failed to fetch World Bank data"}

# ======== 2. EXCHANGE RATE API ========
def fetch_exchange_rates(api_key=None):
    """Fetch exchange rates from ExchangeRate-API or Frankfurter"""
    try:
        if api_key:  # Use ExchangeRate-API if key provided
            url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/USD"
            rates = requests.get(url).json().get("conversion_rates", {})
        else:  # Fallback to Frankfurter (free)
            url = "https://api.frankfurter.app/latest?from=USD"
            rates = requests.get(url).json().get("rates", {})
        return rates if rates else {"error": "No exchange rate data"}
    except Exception as e:
        return {"error": f"Exchange API error: {str(e)}"}

# ======== 3. ALPHA VANTAGE API (STOCK/CRYPTO) ========
def fetch_alpha_vantage(symbol, api_key, function="TIME_SERIES_DAILY"):
    """Fetch stock/crypto data from Alpha Vantage API"""
    url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if "Error Message" in data:
        return {"error": data["Error Message"]}
    return data

# ======== 4. KAGGLE API DATA FUNCTIONS ========
def fetch_kaggle_datasets(search_term="exchange rate", max_results=5):
    """Fetch datasets from Kaggle API"""
    try:
        api = KaggleApi()
        api.authenticate()
        datasets = api.dataset_list(search=search_term, max_size=max_results)
        return [{"title": ds.title, "ref": ds.ref, "size": ds.totalBytes} for ds in datasets]
    except Exception as e:
        return {"error": f"Kaggle API error: {str(e)}"}

def download_kaggle_dataset(dataset_ref, path="./"):
    """Download a specific dataset from Kaggle"""
    try:
        api = KaggleApi()
        api.authenticate()
        api.dataset_download_files(dataset_ref, path=path, unzip=True)
        return {"success": f"Dataset {dataset_ref} downloaded to {path}"}
    except Exception as e:
        return {"error": f"Download error: {str(e)}"}

# ======== EXECUTE ALL APIS ========
if __name__ == "__main__":
    # Setup Kaggle API first
    kaggle_api = setup_kaggle_api()

    print("\n" + "="*60)
    print("=== 🌍 WORLD BANK DATA ===")
    world_bank_data = fetch_world_bank_data(WORLD_BANK_COUNTRY, WORLD_BANK_INDICATOR)
    if "error" not in world_bank_data:
        print(f"{world_bank_data['country']} {world_bank_data['indicator']} ({world_bank_data['year']}): {world_bank_data['value']}")
    else:
        print(world_bank_data["error"])

    print("\n=== 💱 EXCHANGE RATES (USD BASE) ===")
    exchange_rates = fetch_exchange_rates(EXCHANGE_API_KEY)
    if "error" not in exchange_rates:
        df_rates = pd.DataFrame.from_dict(exchange_rates, orient="index", columns=["Rate"])
        display(df_rates.head(5))  # Show top 5 currencies
    else:
        print(exchange_rates["error"])

    print("\n=== 📈 ALPHA VANTAGE STOCK/CRYPTO DATA ===")
    stock_data = fetch_alpha_vantage(STOCK_SYMBOL, ALPHA_VANTAGE_API_KEY)
    if "error" not in stock_data:
        if "Time Series (Daily)" in stock_data:  # Stocks
            df_stock = pd.DataFrame.from_dict(stock_data["Time Series (Daily)"], orient="index")
            df_stock = df_stock.astype(float)
            print(f"Latest {STOCK_SYMBOL} stock data:")
            display(df_stock.head())
        elif "Time Series (Digital Currency Daily)" in stock_data:  # Crypto
            df_crypto = pd.DataFrame.from_dict(stock_data["Time Series (Digital Currency Daily)"], orient="index")
            print(f"Latest {STOCK_SYMBOL} crypto data:")
            display(df_crypto.head())
    else:
        print(stock_data["error"])

    print("\n=== 📊 KAGGLE DATASETS ===")
    if kaggle_api:
        kaggle_datasets = fetch_kaggle_datasets("exchange rate", 3)
        if "error" not in kaggle_datasets:
            print("Available exchange rate datasets:")
            for i, dataset in enumerate(kaggle_datasets, 1):
                print(f"{i}. {dataset['title']} ({dataset['ref']}) - Size: {dataset['size']} bytes")
        else:
            print(kaggle_datasets["error"])
    else:
        print("Kaggle API not available - authentication failed")

    print("\n" + "="*60)
    print("🎉 All API data fetching completed!")

=== 🔧 KAGGLE API SETUP ===
Please upload your kaggle.json file:


Saving kaggle(1).json to kaggle(1).json
Successfully uploaded file: "kaggle(1).json"
Successfully configured the Kaggle API key.

Attempting to authenticate...

❌ Authentication Failed: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

=== 🌍 WORLD BANK DATA ===
United States GDP (current US$) (2022): $26,006,893,000,000.00

=== 💱 EXCHANGE RATES (USD BASE) ===


,Rate
USD,1.0000
AED,3.6725
AFN,68.3263
ALL,83.1316
AMD,383.1999



=== 📈 ALPHA VANTAGE STOCK/CRYPTO DATA ===
Latest IBM stock data:


,1. open,2. high,3. low,4. close,5. volume
2025-08-15,237.61,240.6200,236.77,239.72,4344322.0
2025-08-14,238.25,239.0000,235.62,237.11,4556725.0
2025-08-13,236.20,240.8411,236.20,240.07,5663562.0
2025-08-12,236.53,237.9600,233.36,234.77,8800597.0
2025-08-11,242.24,243.1500,234.70,236.30,9381960.0



=== 📊 KAGGLE DATASETS ===
Kaggle API not available - authentication failed

🎉 All API data fetching completed!
